I can do all this through him who gives me strength

## Selenium

Web Page : http://stats.nba.com/teams/traditional/?sort=PTS&dir=-1

In [2]:
from selenium import webdriver
import requests, json

In [41]:
driver = webdriver.Chrome()
try:
    driver.get('http://stats.nba.com/teams/traditional/?sort=PTS&dir=-1')
    
    # Table Header
    headers = driver.find_elements_by_css_selector("div.nba-stat-table__overflow table > thead > tr > th")
    
    ls = []
    ls.append("RANK")
    for header in headers:
        header = header.text
        if header:
            ls.append(header)
    
    df = pd.DataFrame(columns=ls)
    
    # Table Body
    teams = driver.find_elements_by_css_selector("div.nba-stat-table__overflow table > tbody > tr")
    for team in teams:
        stats = team.find_elements_by_css_selector("td")
        dic = {}
        for idx, data in enumerate(stats):
            row = dic[ls[idx]] = data.text
            
        df.loc[len(df)] = dic
                                           
except Exception as e:
    print(e)
    pass


df.tail()

,RANK,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
25,26,San Antonio Spurs,63,36,27,.571,48.0,102.4,38.9,84.8,...,34.2,44.4,22.9,13.5,7.4,5.6,3.6,17.0,19.6,3.0
26,27,Dallas Mavericks,64,19,45,.297,48.2,102.2,37.9,85.2,...,32.8,40.6,22.6,12.4,7.1,3.8,4.4,19.4,20.2,-2.9
27,28,Miami Heat,63,33,30,.524,48.2,101.4,37.9,83.7,...,33.7,42.9,21.9,14.7,7.3,5.2,4.7,19.9,19.3,-0.6
28,29,Sacramento Kings,64,20,44,.313,48.2,99.2,38.9,85.8,...,31.0,40.4,21.3,14.1,8.3,4.1,4.6,19.8,17.5,-7.8
29,30,Memphis Grizzlies,62,18,44,.290,48.1,98.9,36.2,81.9,...,31.3,40.7,21.3,15.0,7.5,5.0,5.3,23.6,20.8,-4.7


### header

In [ ]:
driver = webdriver.Chrome()
driver.get('http://stats.nba.com/teams/traditional/?sort=PTS&dir=-1')
headers = driver.find_elements_by_css_selector("div.nba-stat-table__overflow table > thead > tr > th")
    
ls = []
ls.append("RANK")
for header in headers:
    header = header.text
    ls.append(header)
    
    if header != "":
        print(header)
    
df = pd.DataFrame(columns=ls)
#print(headers)

In [29]:
df

,RANK,,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,...,,,,,,,,,,


55 columns : Hidden data field까지 포함된 숫자

In [43]:
headers = driver.find_elements_by_css_selector("div.nba-stat-table__overflow table > thead > tr > th")
    
ls = []
ls.append("RANK")
for header in headers:
    header = header.text
    if header:
        ls.append(header)

df = pd.DataFrame(columns=ls)
df

,RANK,TEAM,GP,W,L,WIN%,MIN,PTS,FGM,FGA,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-


28 columns : 보여지는 Header 숫자

### body

In [20]:
teams = driver.find_elements_by_css_selector("div.nba-stat-table__overflow table > tbody > tr")

In [21]:
len(trs)

30

30개 팀에 대한 stats이 30rows에 기록되어 있다.

동적인 페이지여서, Sort by에 따라 Rank가 달라진다.

그래서 Sort by에 따라 index를 다르게 부여하는 방식으로 정렬한다.

In [39]:
stats = team.find_elements_by_css_selector("td")
len(stats)

28

1팀에 28개의 Stats이 담겨있다.

## Selenium -> Json Parsing

In [45]:
driver = webdriver.Chrome()
try:
    driver.get("http://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=")
    response = driver.find_element_by_css_selector("pre").text
    json_data = json.loads(response)
    
    columns = json_data["resultSets"][0]["headers"]
    rows = json_data["resultSets"][0]["rowSet"]
    
    df = pd.DataFrame(rows, columns=columns)
    
except Exception as e:
    print(e)
    pass

driver.close()
df.tail()

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
25,1610612758,Sacramento Kings,64,20,44,0.313,48.2,38.9,85.8,0.453,...,15,7,26,11,14,30,29,29,10,Sacramento Kings
26,1610612759,San Antonio Spurs,63,36,27,0.571,48.0,38.9,84.8,0.459,...,5,20,3,2,1,20,26,5,10,San Antonio Spurs
27,1610612761,Toronto Raptors,62,45,17,0.726,48.2,41.2,87.4,0.472,...,4,8,2,16,27,17,4,2,10,Toronto Raptors
28,1610612762,Utah Jazz,63,33,30,0.524,48.2,37.7,82.7,0.456,...,17,3,15,8,12,14,25,11,10,Utah Jazz
29,1610612764,Washington Wizards,64,36,28,0.563,48.3,40.0,85.8,0.467,...,16,13,22,10,25,12,11,12,10,Washington Wizards


### Hidden Data Fields Filter

In [47]:
columns_filter = ["TEAM_NAME","GP","W","L","W_PCT","MIN","FGM","FGA","FG_PCT","FG3M","FG3A",
                  "FG3_PCT","FTM","FTA","FT_PCT","OREB","DREB","REB","AST","TOV","STL","BLK","BLKA",
                  "PF","PFD","PTS","PLUS_MINUS"]
columns_rename = {"TEAM_NAME":"TEAM", "W_PCT":"WIN%"}

In [48]:
df_filter = df.filter(items=columns_filter)
df_filter.tail()

,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS
25,Sacramento Kings,64,20,44,0.313,48.2,38.9,85.8,0.453,9.0,...,40.4,21.3,14.1,8.3,4.1,4.6,19.8,17.5,99.2,-7.8
26,San Antonio Spurs,63,36,27,0.571,48.0,38.9,84.8,0.459,8.7,...,44.4,22.9,13.5,7.4,5.6,3.6,17.0,19.6,102.4,3.0
27,Toronto Raptors,62,45,17,0.726,48.2,41.2,87.4,0.472,11.6,...,43.5,23.6,13.2,8.2,6.1,4.9,22.1,20.1,111.9,8.6
28,Utah Jazz,63,33,30,0.524,48.2,37.7,82.7,0.456,10.8,...,42.3,21.4,14.5,8.7,4.8,4.5,19.6,20.2,102.8,1.5
29,Washington Wizards,64,36,28,0.563,48.3,40.0,85.8,0.467,9.8,...,43.0,24.6,14.4,8.0,4.4,4.6,21.7,20.3,107.0,1.5


In [49]:
df_rename = df_filter.rename(columns=columns_rename)
df_rename.tail()

,TEAM,GP,W,L,WIN%,MIN,FGM,FGA,FG_PCT,FG3M,...,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS
25,Sacramento Kings,64,20,44,0.313,48.2,38.9,85.8,0.453,9.0,...,40.4,21.3,14.1,8.3,4.1,4.6,19.8,17.5,99.2,-7.8
26,San Antonio Spurs,63,36,27,0.571,48.0,38.9,84.8,0.459,8.7,...,44.4,22.9,13.5,7.4,5.6,3.6,17.0,19.6,102.4,3.0
27,Toronto Raptors,62,45,17,0.726,48.2,41.2,87.4,0.472,11.6,...,43.5,23.6,13.2,8.2,6.1,4.9,22.1,20.1,111.9,8.6
28,Utah Jazz,63,33,30,0.524,48.2,37.7,82.7,0.456,10.8,...,42.3,21.4,14.5,8.7,4.8,4.5,19.6,20.2,102.8,1.5
29,Washington Wizards,64,36,28,0.563,48.3,40.0,85.8,0.467,9.8,...,43.0,24.6,14.4,8.0,4.4,4.6,21.7,20.3,107.0,1.5
